<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/LangChain/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [96]:
!pip install langchain_community langchain_text_splitters langchain_chroma langchain_openai langchain_core langchain_community pypdf

In [97]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [98]:
loader = PyPDFLoader("/content/Sumit_Bandyopadhyay_Resume_v1.0.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [99]:
pages

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-05-13T08:55:39+00:00', 'title': 'Copy of Black White Minimalist CV Resume', 'moddate': '2025-05-13T08:55:35+00:00', 'keywords': 'DAGnUM-cO5w,BACr4bwS9e8,0', 'author': 'sumit', 'source': '/content/Sumit_Bandyopadhyay_Resume_v1.0.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Languages\nC, C++, Python,Rust\nProtocols\nCAN, LIN, J1939, I2C,\nSPI,RS232, TCP/IP suite\nMicro-controller\nKnowledge of 8/16/32 bit\nmicrocontrollers\nOperating System\nOSEK, eCos, uCos, Linux\nDevelopment Tools\nGNUmake tools, Eclipse IDE, Keil\nuVision, MPLab   Microchip,\nCodewarrior, CrossCore\nEmbedded Studio, Visua Dsp++,\nCCS\nConfiguration Tools\nSVN, CM Synergy, Change\nSynergy, Git\nRequirements Tools  \nTelelogic Doors 8.2t\nProject Management Tool \nAgile Jira, MS Project , PMSmart\nTool\nAutomotive Domain Tools\nAutosar Tools Arctic Core,Vector\nCanalyzer, CANoe, DaVinci\nToolchain \nTechnical Skil

In [137]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    separators = ["\n\n","\n","."]
)

docs = text_splitter.split_documents(pages)

In [138]:
docs

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-05-13T08:55:39+00:00', 'title': 'Copy of Black White Minimalist CV Resume', 'moddate': '2025-05-13T08:55:35+00:00', 'keywords': 'DAGnUM-cO5w,BACr4bwS9e8,0', 'author': 'sumit', 'source': '/content/Sumit_Bandyopadhyay_Resume_v1.0.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Languages\nC, C++, Python,Rust\nProtocols\nCAN, LIN, J1939, I2C,\nSPI,RS232, TCP/IP suite\nMicro-controller\nKnowledge of 8/16/32 bit\nmicrocontrollers\nOperating System\nOSEK, eCos, uCos, Linux\nDevelopment Tools\nGNUmake tools, Eclipse IDE, Keil\nuVision, MPLab   Microchip,\nCodewarrior, CrossCore\nEmbedded Studio, Visua Dsp++,\nCCS\nConfiguration Tools\nSVN, CM Synergy, Change\nSynergy, Git\nRequirements Tools  \nTelelogic Doors 8.2t\nProject Management Tool \nAgile Jira, MS Project , PMSmart\nTool'),
 Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-05-13T08:55:39+00:00', 'tit

In [139]:

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [140]:
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})

In [141]:

llm = ChatOpenAI(temperature=0,model="gpt-4o-mini")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

In [160]:
query ="List all companies he has worked in?"

In [161]:

retrieved_docs = retriever_from_llm.invoke(query)


In [162]:
retrieved_docs

[Document(id='0be699f5-bde0-40ce-b3c4-fe6674ac88d0', metadata={'author': 'sumit', 'creationdate': '2025-05-13T08:55:39+00:00', 'creator': 'Canva', 'keywords': 'DAGnUM-cO5w,BACr4bwS9e8,0', 'moddate': '2025-05-13T08:55:35+00:00', 'page': 0, 'page_label': '1', 'producer': 'Canva', 'source': '/content/Sumit_Bandyopadhyay_Resume_v1.0.pdf', 'title': 'Copy of Black White Minimalist CV Resume', 'total_pages': 2}, page_content='2002BE Electronics , Terna Engineering College, Mumbai University\n1998Higher Secondary Board from Atomic Energy Junior College, Mumbai, CBSE\n1996Secondary School Board from Atomic Energy Central School, CBSE\nCertifications\nExperience\nKey Competencies\nDomain Skills Experience on  execution of Full life cycle projects with Aspice L2 compliance\nWorked on Valeo mild hybrid products like  iBSG, Inverters, DC-DC converters, and GMG\nExperience on Off-Highway Vehicle software development'),
 Document(id='b9782420-82a9-4d8f-a8f5-3c71511a76f7', metadata={'author': 'sumit',

In [163]:
template = """You are good at providing the content and then summarizing.
So, please use the below context and summarize without missing any pointers
context: {context}
---------
query: {query}
"""
prompt = PromptTemplate.from_template(template)

In [164]:
model = ChatOpenAI(temperature=0.8)

In [165]:
final_context = ""
for doc in retrieved_docs:
    final_context = final_context+doc.page_content

In [166]:
chain = (
    RunnablePassthrough()  # Initial passthrough without direct input
    | (lambda x: {"context": final_context, "query": x})  # Lambda to create dict input
    | prompt
    | model
    | StrOutputParser()
)

In [167]:
res = chain.invoke({"query":query})


In [168]:
print(res)

Companies he has worked in:
1. HCL Technologies
2. Valeo India Pvt Ltd


Laura, her father, and mother were engaged in social conversation around the fireside one evening in December when they were startled by a violent knocking on the door of their cottage. Two individuals who had lost their way and were very cold requested to warm themselves by the fire, to which Laura's father and mother agreed. However, the situation took a distressing turn when it was revealed that an execution was to take place in the house. Overwhelmed by the news, Laura and her family could only sigh and faint on the sofa. Subsequently, they found themselves alone in the room, missing both their family members and valuable banknotes.


# Combining MultiQueryRetriever and RetrievalQA for Question Answering

In [191]:

from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 10,"score_threshold":0.5})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever_from_llm)

query = "count total years of experience in months since he started working?"
result = qa.run(query)

result


'The individual has 21+ years of experience. To convert this to months:\n\n21 years = 21 * 12 = 252 months\n\nIf we consider "21+" to mean at least 21 years and possibly more, we can say the total is at least 252 months. If we assume "21+" means 21 years and a few additional months, the total would be more than 252 months, but without specific additional months, we can only confirm at least 252 months.'